In [1]:
import requests 
import json
import time
import os
import pandas as pd
from sqlalchemy import engine as sql
from IPython import display
import ipywidgets as widget
import matplotlib.pyplot as plt
import math

In [8]:
data = requests.get('https://api.hh.ru/vacancies?text=java').json()
data

{'items': [{'id': '44706219',
   'premium': False,
   'name': 'Junior Java Developer',
   'department': None,
   'has_test': False,
   'response_letter_required': False,
   'area': {'id': '1', 'name': 'Москва', 'url': 'https://api.hh.ru/areas/1'},
   'salary': {'from': 90000, 'to': 140000, 'currency': 'RUR', 'gross': False},
   'type': {'id': 'open', 'name': 'Открытая'},
   'address': None,
   'response_url': None,
   'sort_point_distance': None,
   'published_at': '2021-06-12T11:39:02+0300',
   'created_at': '2021-06-12T11:39:02+0300',
   'archived': False,
   'apply_alternate_url': 'https://hh.ru/applicant/vacancy_response?vacancyId=44706219',
   'insider_interview': None,
   'url': 'https://api.hh.ru/vacancies/44706219?host=hh.ru',
   'alternate_url': 'https://hh.ru/vacancy/44706219',
   'relations': [],
   'employer': {'id': '193245',
    'name': 'INTECH',
    'url': 'https://api.hh.ru/employers/193245',
    'alternate_url': 'https://hh.ru/employer/193245',
    'logo_urls': {'origi

# Собираем данные по специальности

In [14]:

 
def getPage(page = 0):
    params = {
        'text': 'NAME:Аналитик', # Текст фильтра. В имени должно быть слово "Аналитик"
        'area': 1, # Поиск ощуществляется по вакансиям города Москва
        'page': page, # Индекс страницы поиска на HH
        'per_page': 100 # Кол-во вакансий на 1 странице
    }
     
    req = requests.get('https://api.hh.ru/vacancies', params) # Посылаем запрос к API
    data = req.content.decode() # Декодируем его ответ, чтобы Кириллица отображалась корректно
    req.close()
    return data
 
for page in range(0, 20):
    jsObj = json.loads(getPage(page))
    nextFileName = './docs/pagination/{}.json'.format(len(os.listdir('./docs/pagination')))
    f = open(nextFileName, mode='w', encoding='utf8')
    f.write(json.dumps(jsObj, ensure_ascii=False))
    f.close()
    if (jsObj['pages'] - page) <= 1:
        break
     
    # Необязательная задержка, но чтобы не нагружать сервисы hh, оставим. 5 сек мы может подождать
    time.sleep(0.25)
     
print('Старницы поиска собраны')

Старницы поиска собраны


# Собираем данные по вакансии

In [16]:
 
# Получаем перечень ранее созданных файлов со списком вакансий и проходимся по нему в цикле 
for fl in os.listdir('./docs/pagination'):
     
    # Открываем файл, читаем его содержимое, закрываем файл
    f = open('./docs/pagination/{}'.format(fl), encoding='utf8')
    jsonText = f.read()
    f.close()
     
    # Преобразуем полученный текст в объект справочника
    jsonObj = json.loads(jsonText)
     
    # Получаем и проходимся по непосредственно списку вакансий
    for v in jsonObj['items']:
         
        # Обращаемся к API и получаем детальную информацию по конкретной вакансии
        req = requests.get(v['url'])
        data = req.content.decode()
        req.close()
         
        # Создаем файл в формате json с идентификатором вакансии в качестве названия
        # Записываем в него ответ запроса и закрываем файл
        fileName = './docs/vacancies/{}.json'.format(v['id'])
        f = open(fileName, mode='w', encoding='utf8')
        f.write(data)
        f.close()
         
        time.sleep(0.25)
         
print('Вакансии собраны')

Вакансии собраны


# Собираем данные в  базу данных

In [24]:

 
# Создаем списки для столбцов таблицы vacancies
IDs = [] # Список идентификаторов вакансий
names = [] # Список наименований вакансий
descriptions = [] # Список описаний вакансий
 
# Создаем списки для столбцов таблицы skills
skills_vac = [] # Список идентификаторов вакансий
skills_name = [] # Список названий навыков
 
# В выводе будем отображать прогресс
# Для этого узнаем общее количество файлов, которые надо обработать
# Счетчик обработанных файлов установим в ноль
cnt_docs = len(os.listdir('./docs/vacancies'))
i = 0
 
# Проходимся по всем файлам в папке vacancies
for fl in os.listdir('./docs/vacancies'):
     
    # Открываем, читаем и закрываем файл
    f = open('./docs/vacancies/{}'.format(fl), encoding='utf8')
    jsonText = f.read()
    f.close()
     
    # Текст файла переводим в справочник
    jsonObj = json.loads(jsonText)
     
    # Заполняем списки для таблиц
    IDs.append(jsonObj['id'])
    names.append(jsonObj['name'])
    descriptions.append(jsonObj['description'])
     
    # Т.к. навыки хранятся в виде массива, то проходимся по нему циклом
    for skl in jsonObj['key_skills']:
        skills_vac.append(jsonObj['id'])
        skills_name.append(skl['name'])
     
    # Увеличиваем счетчик обработанных файлов на 1, очищаем вывод ячейки и выводим прогресс
    i += 1
    display.clear_output(wait=True)
    display.display('Готово {} из {}'.format(i, cnt_docs))
 
 
# Создадим соединение с БД
eng = sql.create_engine('localdatabase')
conn = eng.connect()
 
# Создаем пандосовский датафрейм, который затем сохраняем в БД в таблицу vacancies
df = pd.DataFrame({'id': IDs, 'name': names, 'description': descriptions})
df.to_sql('vacancies', conn, schema='public', if_exists='append', index=False)
 
# Тоже самое, но для таблицы skills
df = pd.DataFrame({'vacancy': skills_vac, 'skill': skills_name})
df.to_sql('skills', conn, schema='public', if_exists='append', index=False)
 
# Закрываем соединение с БД
conn.close()
 
# Выводим сообщение об окончании программы
display.clear_output(wait=True)
display.display('Вакансии загружены в БД')

'Вакансии загружены в БД'

# Создаем облако вакансий из токенов

In [37]:

conn = sqlalchemy.create_engine('localdatabase').connect()
sql = 'select name from public.vacancies'
vacancies_name = pd.read_sql(sql, conn).name
sql = "select v.name,skill from public.skills s join public.vacancies v on v.id = s.vacancy"
skills = pd.read_sql(sql, conn)

conn.close()

from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer

cv=CountVectorizer(ngram_range=(2,2),stop_words=['ведущий', 'главный', 'младший', 'эксперт', 'стажер', 'старший', 'middle', 'junior', 'senior'],min_df=0.001)
word_vector=cv.fit_transform(vacancies_name)

idf = TfidfTransformer().fit(word_vector)

df_idf = pd.DataFrame(idf.idf_, index=cv.get_feature_names(), columns=["idf"])

pivot = pd.DataFrame(word_vector.toarray(), columns=cv.get_feature_names(), index=vacancies_name.values)

In [38]:
# Импортируем виджеты. Они позволяют более гибко управлять выводом jupyter и 
# создавать интерактивные элементы
import ipywidgets as widget
 
# Импортируем модуль вывода jupyter
from IPython import display
 
# Библиотека для визуализации данных
import matplotlib.pyplot as plt
 
# Модуль математических функций
import math
 
################################################
 
def click_back(b):
    """
       Метод, который вызывается при клике на кнопку "Назад".
       Метод скрывает информацию о перечне вакансий из левого сайдбара и
       диаграмму популярности навыков из правого сайдбара, а также отображает
       в центральной части шаблона облако токенов.
    """
    app_layout.left_sidebar = None
    app_layout.right_sidebar = None
    app_layout.center = out_words
 
################################################
 
def print_words(x):
    """
        Метод, который вызывается при изменении значения слайдера.
        Метод отрисовывает в формате html облако токенов
         
        Аргумент "x" принимает справочник, который содержит состояние слайдера. Используем
        x.new, чтобы получить новое состояние слайдера.
    """
     
    # Получаем датафрейм, отсортированный в обратном порядке по мере idf, 
    # ограниченный диапазоном значений, заданным слайдером
    ds = df_idf.sort_values(by='idf', ascending=False)[x.new[0]: x.new[1]]
     
    # Получаем максимальное и минимальное значения idf. Будем использовать их для установки
    # размера шрифта токена в облаке
    mx = df_idf['idf'].max()
    mn = df_idf['idf'].min()
     
    # Задаем общие css-стили для облака токенов
    tags = """<style>
    .tagword{
        border:1px solid #bee5eb;
        padding:1px 5px;
        display:block;
        border-radius:4px;
        color:#0c5460;
        background:#d1ecf1;
        line-height:normal;
        cursor:pointer}
    .tagword:hover{
        background:#c1dce1}
    .tag-wrapper{
        float:left;
        margin:0 5px 5px 0}
    </style>"""
     
    # Пробегаемся по токенам, выбранным с помощью слайдера
    for r in ds.sort_index().itertuples():
         
        # Масштабируем значения idf от 0 до 1 и переворачиваем их, 
        # чтобы максимальное значение стало минимальным.
        if mx > mn:
             
            # Задаем размер шрифта и высоту токена в облаке
            fs = int( (((r.idf - mn) / (mx - mn)) * -1 + 1 ) * 30 + 10 )
            hd = math.ceil(fs / 10) * 10 + 8
        else:
            fs = 40
         
        # Добавляем токен в облако в формате html. Токену назначаем функцию tag_click для
        # события клика. Сама функция описана ниже за пределами данного метода
        tag_tmpl = """<div class="tag-wrapper" style="height:{height}px">
            <span onclick="tag_click(this)" class="tagword" style="font-size:{size}px">{name}</span>
        </div>"""
        tags += tag_tmpl.format(name=r.Index, size=fs, height=hd)
     
    # Отрисовываем облако токенов 
    click_back(None)
    out_words.clear_output(wait=True)
    with out_words:
        display.display_html(tags, raw=True)
         
################################################
         
def get_detail(w):
    """
        Метод отрисовывает детелизацию по токену, а именно перечень вакансий, 
        содержащих переданный токен, и топ-20 самых востребованных навыков по этим вакансиям.
         
        Аргумент "w" принимает значение токена, по которому кликнули
    """
     
    # Выводим пандосовский датафрейм со списком вакансий, содержащих токен
    out_details_vac.clear_output(wait=True)
    vacs = pivot[pivot[w] > 0].sort_index().index.unique()
    with out_details_vac:
        display.display_html(
                pd.DataFrame(data=vacs, columns=['Список вакансий:']).to_html(index=False), raw=True
        )
     
    # Выводим горизонтальный столбчатый график самых популярных навыков по выбранным вакансиям
    out_details_skl.clear_output(wait=True)
    skill_pvt = skills \
                .query('name in @vacs') \
                .groupby(by='skill', as_index=False) \
                .agg({'name':'count'}) \
                .sort_values(by='name', ascending=False).head(20)
    fig_h = skill_pvt.shape[0] / 2.5    # Корректируем размер графика на 
                                        # основании количества получившихся столбцов
    with out_details_skl:
        skill_pvt.sort_values(by='name').plot.barh(
                        x='skill', 
                        y='name', 
                        figsize=(5, fig_h), 
                        legend=None, 
                        title='ТОП20 Навыков'
        )
        plt.show()
     
     
    # Скрываем облако токенов и выводим детализацию для токена
    app_layout.center = None
    app_layout.left_sidebar = out_details_vac
    app_layout.right_sidebar = out_details_skl
 
################################################
 
# Создаем несколько виджетов вывода
out_header = widget.Output() # Сюда будем выводить виджет слайдера и кнопку возврата
out_words = widget.Output() # Сюда будем выводить облако
out_details_vac = widget.Output() # Сюда будем выводить перечень вакансий, содержащих токен
out_details_skl = widget.Output() # Сюда будем выводить график навыков по вакансиям, содержащим токен
 
# Создаем виджет макета. Весь отчет по сути располагается в данном макете.
# Пока только с выводом в центр виджета с облаком
app_layout = widget.AppLayout(
    header=None,
    left_sidebar=None,
    center=out_words,
    right_sidebar=None,
    footer=None,
    pane_heights=[1, 5, '60px']
)
 
# Создаем виджет слайдера, для указания диапазона рассматриваемых токенов
sld = widget.IntRangeSlider(
    value=[0, 0],
    min=0,
    max=df_idf['idf'].count(),
    step=1,
    description='Частота:',
    layout = {'width': '100%'}
)
# Назначаем слайдеру метод, который будет вызываться при изменении значения слайдера
sld.observe(print_words, names='value')
 
# Создаем виджет кнопки
bck_btn = widget.Button(
    description='Назад',
    button_style='info'
)
# Назначаем кнопке метод, который будет вызываться по событию клика по ней
bck_btn.on_click(click_back)
 
# В формате html подготовим строку, которая создаст функцию на языке javascript
# Данную функцию будем вызывать по событию клика по токену в облаке
html = """
<script>
    function tag_click(e){
        var kernel = Jupyter.notebook.kernel
        func = 'get_detail("' + e.innerText + '")'
        kernel.execute(func)
    }
</script>"""
 
# Выводим в виджет вывода виджеты слайдера и кнопки
with out_header:
    display.display(sld)
    display.display(bck_btn)
 
display.display_html(html, raw=True) # Отображаем в ячейке ранее подготовленный html c js-функцией
display.display(out_header) # Отображаем виджет, содержащий слайдер и кнопку
display.display(app_layout) # Отображаем виджет макета
 
# Задаем новые значения диапазона слайдера, чтобы отобразить ТОП-30 самых популярных токенов
# Данная строка спровоцирует изменения, что в свою очередь вызовет установленный метод print_words,
# который формирует облако
sld.value = [sld.max - 30, sld.max]

Output()

AppLayout(children=(Output(layout=Layout(grid_area='center')),), layout=Layout(grid_template_areas='"center ce…